# Business Insight - Customer Use

This notebook aims at quantifying product use for each customer. The information is fundamental for Task 3

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
from tqdm import tqdm_notebook
import pickle

## Active devices per user

How many devices does each user use each day?

In [6]:
# import connected devices dictionary
conndev = pickle.load( open('connected_devices_1day.pk','rb'))

In [7]:
# import device-user database
device_user_db = pd.read_csv('device_user_database.csv')

### Functions

In [8]:
def device_array_to_user_device_count(connarr, device_user_db):
    # returns a DataFrame with user_id and device count from an array with device_id's
    
    inp_df = pd.DataFrame({'device_id': connarr})
    # join tables
    inp_df = inp_df.join(device_user_db.set_index('device_id'), on='device_id', how='left')
    dc = inp_df.groupby('user_id')['device_id'].count().to_frame()
    return dc

In [10]:
def connected_dict_to_user_device_count( conn_dict, device_user_db ):
    # loops over an entire connected device dictionary producing
    # a dataframe with device count per user per day
    
    for k, (key, array) in enumerate( tqdm_notebook( conn_dict.items())):
        # get the count segmented by fleet size
        dfcat = device_array_to_user_device_count(array,device_user_db)
        # rename column
        dfcat.columns=[key]
        if k==0:
            res = dfcat
        else:
            res = pd.merge(res,dfcat,left_index=True,right_index=True, how='outer')
        
    return res

### Run

In [11]:
res = connected_dict_to_user_device_count(conndev, device_user_db)

In [12]:
# set NaN's to 0
res[res.isnull()]=0

In [14]:
# transpose in order to have dates as index
res = res.transpose()

In [15]:
res.shape

(1005, 5455)

In [16]:
# set index as datetime objects
res.index = pd.date_range(datetime(2015,5,31),datetime(2018,2,28))

In [17]:
res.to_csv('Active_Devices_per_user_per_day.csv',index=True)